In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

# from __future__ import absolute_import, division, print_function
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime



import rpy2
import rpy2.robjects.numpy2ri
from stldecompose import decompose
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
stats = importr('stats')
stl=stats.stl
ts =stats.ts

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data= pd.read_csv("Data/FromMohammed/cif_dataset_complete.csv",header=None)
predictions = pd.read_csv("Data/FromMohammed/theta_25_horg.csv",index_col=0,skiprows = [1])

In [3]:
print(data.shape)
data.head(5)

(72, 123)


,0,1,2,3,4,5,6,7,8,9,...,113,114,115,116,117,118,119,120,121,122
0,ts1,12,monthly,855.144866,847.292062,824.731660,866.645406,928.333300,927.649280,901.072255,...,1585.222987,1674.195284,1674.955086,1695.759843,1655.823053,1672.327141,1650.605735,1738.656214,1710.378030,1683.418022
1,ts2,12,monthly,496.999629,491.266202,514.497083,505.818719,523.568994,525.329842,529.642170,...,720.558589,743.722838,726.890762,739.548510,744.333120,740.740383,752.161177,756.418571,758.655439,768.512031
2,ts3,12,monthly,1333.548021,1370.089237,1415.076738,1281.416912,1358.059752,1384.964713,1276.752289,...,792.898046,732.690151,773.346272,782.951732,749.658718,784.576004,786.650821,672.321333,637.252308,725.853345
3,ts4,12,monthly,601.167280,541.750384,601.225895,561.288487,543.068962,560.192329,585.670783,...,1019.850913,993.553967,1000.544495,981.014694,989.564612,1044.511185,1095.874066,1117.322564,1106.108627,1066.770728
4,ts5,12,monthly,507.766151,498.915098,513.571657,507.190113,510.092496,506.453491,514.185068,...,577.076351,566.641173,604.329118,580.958679,637.388618,635.713882,634.583760,691.642323,710.045035,739.101974


In [4]:
print(predictions.shape)
predictions.head(5)

(72, 90)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
2,1081.444132,1087.148239,1092.882433,1098.646871,1104.441714,1110.267123,1116.123257,1122.010280,1127.928354,1133.877643,...,1636.329309,1642.124872,1647.940962,1653.777651,1664.145022,1669.711184,1675.295964,1680.899424,1686.521626,1692.162632
3,561.950940,563.277718,564.602934,565.926597,567.248714,568.569291,569.888337,571.205857,572.521860,573.836351,...,722.343037,723.748655,725.157008,726.568101,735.430566,736.659927,737.888263,739.115577,740.341873,741.567154
4,1179.646121,1175.349049,1171.051976,1166.754904,1162.457831,1158.160758,1153.863686,1149.566613,1145.269541,1140.972468,...,791.032385,787.828474,784.644752,781.481090,764.614275,761.654324,758.711887,755.786861,752.879143,749.988631
5,592.584192,593.290366,593.997825,594.706570,595.416605,596.127931,596.840552,597.554468,598.269683,598.986200,...,917.018262,922.150443,927.311347,932.501135,1006.294718,1014.192403,1022.182148,1030.265025,1038.442121,1046.714532
6,511.011338,511.094677,511.178030,511.261396,511.344776,511.428170,511.511578,511.594998,511.678433,511.761881,...,531.546767,531.940904,532.335042,532.729179,586.182927,587.642310,589.107559,590.578696,592.055747,593.538733


In [5]:
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
#     depth=data.shape[2]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size+1-horizon):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def make_k_input(data,window_size,horizon):
    length = data.shape[0]
    output= np.zeros([length-window_size+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
#     print(str(extra))
    data = np.append(data,np.zeros([horizon-extra]))
#     print(data)
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_k_input(data,window_size,horizon):
    length = data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    
#     data = np.append(data,np.zeros([horizon-extra]))
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)    

In [6]:
#def sMAPE(val,preds,horizon):
#    temp = np.abs(val-preds)
#    temp1=np.abs(val)+np.abs(preds)    
#    smape = 2/horizon*np.sum(temp/temp1)
#    return smape

#def sMAPE1(val,preds):
#    temp = np.abs(val-preds)
#    temp1=np.abs(val)+np.abs(preds)    
#    smape = 2/np.sum(temp/temp1)
#    return smape

#def smape(A, F):
#    return 1/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


##def smape(y_true, y_pred):   #http://shortnotes.herokuapp.com/how-to-implement-smape-function-in-python-149
##    denominator = (np.abs(y_true) + np.abs(y_pred))
##    diff = np.abs(y_true - y_pred) / denominator
##    diff[denominator == 0] = 0.0
##    return 200 * np.mean(diff)

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200.0 * np.mean(diff)

from numba import jit   #https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/37232
import math
@jit
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = np.abs(a)+np.abs(b)
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (2.0 / y_true.shape[0])
    return out



# Case 0:
##### Using only data (no prediction)
##### Using Decomposition Method
##### Using 3Layers Conv (64) Neural Network 

In [8]:
print(datetime.datetime.now())
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            #horizon = data.iloc[y].values[1]
            horizon=6
            window_size=6
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            #rr = current_row.size
            #rr = int(np.floor(rr*.25))
            #temp1=current_row[rr:] # values after (rr)th position in current_row
            temp1=current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            frequency = 7     #check
            result = stl(ts(temp1,frequency=frequency),"periodic")
            result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            series_d  = result[:,0]+result[:,2]
#############################################################################
            #series=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            #current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            #current_pred[np.argwhere(current_pred<=0)]=0.5
            #current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            ##series_pred = result_pred[:,0]+result_pred[:,2]
            #series_p = result_pred[:,0]+result_pred[:,1]+result_pred[:,2]#check
#############################################################################
            ##series_pred=current_pred
            #series_pred=series_p[:-horizon]
            #train_p = series_pred[:-n_val]
            ##val_p = series_pred[-(n_val+window_size+3):]
            #val_p = series_pred[-(2*n_val+window_size):]
            #test_p = series_p[-(horizon+window_size):]

            #train_pred = make_k_input(train_p,window_size,horizon)
            #val_pred = make_k_input(val_p,window_size,horizon)
            #test_pred = nonov_make_k_input(test_p,window_size,horizon)


            ##train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            ##val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            ##test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            #train_input = np.append(x_train,train_pred,axis=1)
            train_input = x_train
            #val_input = np.append(x_val,val_pred,axis=1)
            val_input = x_val
            #test_input = np.append(x_test,test_pred,axis=1)
            test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            #input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_data= Input(batch_shape=(None,window_size,1),name='input_data')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF06.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF06.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]
            
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction06.csv',final_predictions, fmt='%1.3f',delimiter=',')
        smape1=smape(final_predictions,final_test)
        mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)


2019-07-28 18:35:42.126037


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [09:07<00:00, 10.26s/it]


2019-07-28 18:44:49.510077
SMAPE:
150.77399076731973
-----------------
MSE:
0.007831297099438668


# Case 1:
##### Using only data (no prediction)
##### Without Decomposition 
##### Using 3Layers Conv (64) Neural Network 

In [7]:
print(datetime.datetime.now())
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            #horizon = data.iloc[y].values[1]
            horizon =6
            window_size=6
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            #rr = current_row.size
            #rr = int(np.floor(rr*.25))
            #temp1=current_row[rr:] # values after (rr)th position in current_row
            temp1=current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
            ##series_d  = result[:,0]+result[:,1]+result[:,2]#check
#############################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            #current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            #current_pred[np.argwhere(current_pred<=0)]=0.5
            #current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            ##series_pred = result_pred[:,0]+result_pred[:,2]
            #series_p = result_pred[:,0]+result_pred[:,1]+result_pred[:,2]#check
#############################################################################
            ##series_pred=current_pred
            #series_pred=series_p[:-horizon]
            #train_p = series_pred[:-n_val]
            ##val_p = series_pred[-(n_val+window_size+3):]
            #val_p = series_pred[-(2*n_val+window_size):]
            #test_p = series_p[-(horizon+window_size):]

            #train_pred = make_k_input(train_p,window_size,horizon)
            #val_pred = make_k_input(val_p,window_size,horizon)
            #test_pred = nonov_make_k_input(test_p,window_size,horizon)


            ##train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            ##val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            ##test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            #train_input = np.append(x_train,train_pred,axis=1)
            train_input = x_train
            #val_input = np.append(x_val,val_pred,axis=1)
            val_input = x_val
            #test_input = np.append(x_test,test_pred,axis=1)
            test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            #input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_data= Input(batch_shape=(None,window_size,1),name='input_data')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF16.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF16.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction16.csv',final_predictions, fmt='%1.3f',delimiter=',')
        smape1=smape(final_predictions,final_test)
        mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)


2019-07-29 00:13:09.400613


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [07:21<00:00,  9.06s/it]


2019-07-29 00:20:31.259575
SMAPE:
81.03740008205219
-----------------
MSE:
24.62436661589032


# Case 2:
##### Using Decomposition Method
##### Using Combination of Data and Prediction
##### Using 3Layers Conv (64) Neural Network 

In [17]:
print(datetime.datetime.now())
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            #horizon = data.iloc[y].values[1]
            horizon=6
            window_size=6
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            frequency = 7     #check
            result = stl(ts(temp1,frequency=frequency),"periodic")
            result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            series_d  = result[:,0]+result[:,2]
#############################################################################
            #series=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            series_p = result_pred[:,0]+result_pred[:,2]
#############################################################################
            #series_p=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)


            #train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            #val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            #test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = np.append(x_train,train_pred,axis=1)
            #train_input = x_train
            val_input = np.append(x_val,val_pred,axis=1)
            #val_input = x_val
            test_input = np.append(x_test,test_pred,axis=1)
            #test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF26.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF26.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction26.csv',final_predictions, fmt='%1.3f',delimiter=',')
        smape1=smape(final_predictions,final_test)
        mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)


2019-07-28 19:47:24.521588


100%|████████████████████████████████████████████████████████████████████████████████| 72/72 [1:02:30<00:00, 52.16s/it]


2019-07-28 20:49:54.994159
SMAPE:
153.22723128373227
-----------------
MSE:
0.008234016047789419


# Case 3 :
##### Using Raw Data and Prediction (without Decomposition)
##### Using Combination of Data and Prediction
##### Using 3Layers Conv (64) Neural Network 

In [8]:
print(datetime.datetime.now())

data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            #horizon = data.iloc[y].values[1]
            horizon=6
            window_size=6
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
#############################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            #series_p = result_pred[:,0]+result_pred[:,2]
#############################################################################
            series_p=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)


            #train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            #val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            #test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = np.append(x_train,train_pred,axis=1)
            #train_input = x_train
            val_input = np.append(x_val,val_pred,axis=1)
            #val_input = x_val
            test_input = np.append(x_test,test_pred,axis=1)
            #test_input = x_test

############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_pred=Input(batch_shape=(None,horizon),name='input_pred')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF36.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF36.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]

            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction36.csv',final_predictions, fmt='%1.3f',delimiter=',')
        smape1=smape(final_predictions,final_test)
        mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)


2019-07-29 00:20:31.981619


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [19:03<00:00, 19.12s/it]


2019-07-29 00:39:35.797062
SMAPE:
77.60481543537496
-----------------
MSE:
21.430987044145862


# Case 4 :
##### using Combination of Data and Prediction
##### Using Decomposition
##### Using autoencoder on Prediction (6 to 4)
##### Merging Data and Prediction to feed the NN
##### Using 3Layers Conv (64) Neural Network

In [ ]:
print(datetime.datetime.now())
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            window_size=6
            horizon=6
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)
            ########################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
            #################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))

            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

################################################################################
###creating train, validation and test sets from Predictions####################
################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
            ########################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            #series_p = result_pred[:,0]+result_pred[:,2]
            #################################################################
            series_p=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)

################################################################################
############        Auto-Encoder Configuration          ########################
################################################################################
            series_p_auto=train_pred
            series_p2_auto=val_pred
            series_p3_auto=test_pred


            train_pred_auto=np.zeros([series_p_auto.shape[0],4,1])
            val_pred_auto=np.zeros([series_p2_auto.shape[0],4,1])
            test_pred_auto=np.zeros([series_p3_auto.shape[0],4,1])

            ncol = series_p_auto.shape[1]
            encoding_dim = 4
            input_dim = Input(shape = (ncol, ))
            # Encoder Layers
            encoded1 = Dense(5, activation = 'relu')(input_dim)
            encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
            # Decoder Layers
            decoded1 = Dense(5, activation = 'relu')(encoded2)
            decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
            # Combine Encoder and Deocder layers
            autoencoder = Model(inputs = input_dim, outputs = decoded2)
            autoencoder.compile(optimizer = 'adam', loss = 'mse')

            for k in range(series_p_auto.shape[0]):
                data_auto=np.array(series_p_auto[0]).transpose()
                autoencoder.fit(data_auto, data_auto, nb_epoch = 10, batch_size = 2, shuffle = False) 
                encoder = Model(inputs = input_dim, outputs = encoded2)
                encoded_input = Input(shape = (encoding_dim, ))
                auto_out = np.array(encoder.predict(data_auto))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                auto_out=pd.DataFrame(auto_out)
                train_pred_auto[0]=auto_out

            for k in range(series_p2_auto.shape[0]):
                data_auto=np.array(series_p2_auto[0]).transpose()
                autoencoder.fit(data_auto, data_auto, nb_epoch = 10, batch_size = 2, shuffle = False) 
                encoder = Model(inputs = input_dim, outputs = encoded2)
                encoded_input = Input(shape = (encoding_dim, ))
                auto_out = np.array(encoder.predict(data_auto))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                auto_out=pd.DataFrame(auto_out)
                val_pred_auto[0]=auto_out
    
            for k in range(series_p3_auto.shape[0]):
                data_auto=np.array(series_p3_auto[0]).transpose()
                autoencoder.fit(data_auto, data_auto, nb_epoch = 10, batch_size = 2, shuffle = False) 
                encoder = Model(inputs = input_dim, outputs = encoded2)
                encoded_input = Input(shape = (encoding_dim, ))
                auto_out = np.array(encoder.predict(data_auto))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                auto_out=pd.DataFrame(auto_out)
                test_pred_auto[0]=auto_out

################################################################################
############    Creating inputs for Neural Network      ########################
################################################################################
            train_input = np.append(x_train,train_pred_auto,axis=1)
            #train_input = x_train
            val_input = np.append(x_val,val_pred_auto,axis=1)
            #val_input = x_val
            test_input = np.append(x_test,test_pred_auto,axis=1)
            #test_input = x_test

################################################################################
############       Neural Network Configuration         ########################
################################################################################
            input_data= Input(batch_shape=(None,horizon+4,1),name='input_data')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF46.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF46.h5')
            pred=model.predict({'input_data':test_input})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction46.sv',final_predictions, fmt='%1.3f',delimiter=',')
    smape1=smape(final_predictions,final_test)
    mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)

In [ ]:
2+2

# Case 4.5
##### using Combination of Data and Prediction
##### Using Decomposition
##### Using autoencoder on Prediction (6 to 4)
##### Merging Data and Prediction to feed the NN
##### Using 3Layers Conv (64) Neural Network

In [ ]:
2019-07-29 00:07:27.386410
SMAPE:
154.7692360773325
-----------------
MSE:
549120387862.9599

# Case 5 :
##### Using Data and Prediction 
##### Using Decomposition
##### Using Combination of Data and Prediction
##### adding Prediction n the Output of Neural Network
##### Using 3Layers Conv (64) Neural Network 

In [13]:
print(datetime.datetime.now())

data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            #horizon = data.iloc[y].values[1]
            horizon=6
            window_size=6
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            frequency = 7     #check
            result = stl(ts(temp1,frequency=frequency),"periodic")
            result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            series_d  = result[:,0]+result[:,2]
#############################################################################
            #series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            series_p = result_pred[:,0]+result_pred[:,2]
#############################################################################
            #series_p=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)
            
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = np.append(x_train,train_pred,axis=1)
            #train_input = x_train
            val_input = np.append(x_val,val_pred,axis=1)
            #val_input = x_val
            test_input = np.append(x_test,test_pred,axis=1)
            #test_input = x_test

            
            train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_pred=Input(batch_shape=(None,horizon),name='input_pred')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
            net=add([net,input_pred])
            

            model=Model(inputs=[input_data,input_pred],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF56.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input,'input_pred':train_pred },y_train,validation_data=[[val_input,val_pred],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF56.h5')
            pred=model.predict({'input_data':test_input, 'input_pred':test_pred})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]

            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction56.csv',final_predictions, fmt='%1.3f',delimiter=',')
        smape1=smape(final_predictions,final_test)
        mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)


2019-07-29 00:58:23.791175


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [36:25<00:00, 32.44s/it]


2019-07-29 01:34:49.369283
SMAPE:
142.85014588907597
-----------------
MSE:
0.0687628708195608


# Case 6 :
##### Using Data and Prediction 
##### Without Decomposition
##### Using Combination of Data and Prediction
##### adding Prediction n the Output of Neural Network
##### Using 3Layers Conv (64) Neural Network 

In [14]:
print(datetime.datetime.now())

data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            #horizon = data.iloc[y].values[1]
            horizon=6
            window_size=6
            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)

####################################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
#############################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))
            
            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

############################################################################################
###############creating train, validation and test sets from Predictions####################
############################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
####################################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            #series_p = result_pred[:,0]+result_pred[:,2]
#############################################################################
            series_p=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)
            
############################################################################################
########################    Creating inputs for Neural Network      ########################
############################################################################################
            train_input = np.append(x_train,train_pred,axis=1)
            #train_input = x_train
            val_input = np.append(x_val,val_pred,axis=1)
            #val_input = x_val
            test_input = np.append(x_test,test_pred,axis=1)
            #test_input = x_test

            
            train_pred=train_pred.reshape(train_pred.shape[0],horizon)
            val_pred=val_pred.reshape(val_pred.shape[0],horizon)
            test_pred=test_pred.reshape(test_pred.shape[0],horizon)
############################################################################################
########################       Neural Network Configuration         ########################
############################################################################################
            input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
            input_pred=Input(batch_shape=(None,horizon),name='input_pred')

            branch_0 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_1 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            branch_2 = Conv1D(64,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)
            net=add([net,input_pred])
            

            model=Model(inputs=[input_data,input_pred],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF66.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input,'input_pred':train_pred },y_train,validation_data=[[val_input,val_pred],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF66.h5')
            pred=model.predict({'input_data':test_input, 'input_pred':test_pred})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]

            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction66.csv',final_predictions, fmt='%1.3f',delimiter=',')
        smape1=smape(final_predictions,final_test)
        mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)


2019-07-29 01:39:23.059835


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [58:25<00:00, 45.16s/it]


2019-07-29 02:37:48.199286
SMAPE:
3.3065771706903924
-----------------
MSE:
0.15520938863631786


# Case 7 :
##### Using Data and Prediction 
##### Without Decomposition
##### Using autoencoder on prediction (6 to 4)
##### Using 3Layers Conv (32) Neural Network 
##### Adding output of autoencoder inside the Neural Network (different layers)

In [ ]:
print(datetime.datetime.now())

data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    for horizon in [1]:
        final_predictions = np.zeros([data_length,6])
        final_test = np.zeros([data_length,6])
        for y in range(data_length):
############################################################################################
######################creating train, validation and test sets from Data####################
############################################################################################
            #horizon = data.iloc[y].values[1]
            horizon=6
            window_size=6

            current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
            rr = current_row.size
            rr = int(np.floor(rr*.25))
            temp1=current_row[rr:] # values after (rr)th position in current_row
            epsilon = 0.05     #check
            temp1[temp1<epsilon] = temp1[temp1<epsilon]+0.05
            temp1 = np.log(temp1)
            ########################  Decomposition ##########################
            #frequency = 7     #check
            #result = stl(ts(temp1,frequency=frequency),"periodic")
            #result=pandas2ri.ri2py(result.rx2('time.series')) # result[:,0]->seasonal // result[:,1]->trend //  result[:,2]->residual
            #series_d  = result[:,0]+result[:,2]
#################################################################
            series_d=temp1
            series_data = series_d[:-horizon]
            series_length = series_data.size
            n_val = int(np.round(series_length*.2))

            train = series_data[:-n_val]
            test = series_d[-(horizon+window_size):]
            #val = series_data[-(n_val+window_size+3):]
            val = series_data[-(2*n_val+window_size):]

            train_sequence = make_input(train, window_size,horizon)
            val_sequence = make_input(val,window_size,horizon)
            test_sequence = nonov_make_input(test,window_size,horizon)

            x_train = train_sequence[0]
            y_train =train_sequence[1]
            x_val = val_sequence[0]
            y_val = val_sequence[1]
            x_test = test_sequence[0]
            y_test = test_sequence[1]

################################################################################
###creating train, validation and test sets from Predictions####################
################################################################################
            current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
            current_pred[np.argwhere(current_pred<=0)]=0.5
            current_pred = np.log(current_pred)
########################  Decomposition ##########################
            #result_pred = stl(ts(current_pred,frequency=frequency),"periodic")
            #result_pred=pandas2ri.ri2py(result_pred.rx2('time.series'))
            #series_p = result_pred[:,0]+result_pred[:,2]
#################################################################
            series_p=current_pred
            series_pred=series_p[:-horizon]
            train_p = series_pred[:-n_val]
            #val_p = series_pred[-(n_val+window_size+3):]
            val_p = series_pred[-(2*n_val+window_size):]
            test_p = series_p[-(horizon+window_size):]

            train_pred = make_k_input(train_p,window_size,horizon)
            val_pred = make_k_input(val_p,window_size,horizon)
            test_pred = nonov_make_k_input(test_p,window_size,horizon)

################################################################################
############        Auto-Encoder Configuration          ########################
################################################################################
            series_p_auto=train_pred
            series_p2_auto=val_pred
            series_p3_auto=test_pred


            train_pred_auto=np.zeros([series_p_auto.shape[0],32,1])
            val_pred_auto=np.zeros([series_p2_auto.shape[0],32,1])
            test_pred_auto=np.zeros([series_p3_auto.shape[0],32,1])

            ncol = series_p_auto.shape[1]
            encoding_dim = 4
            input_dim = Input(shape = (ncol, ))
            # Encoder Layers
            encoded1 = Dense(5, activation = 'relu')(input_dim)
            encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
            # Decoder Layers
            decoded1 = Dense(5, activation = 'relu')(encoded2)
            decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
            # Combine Encoder and Deocder layers
            autoencoder = Model(inputs = input_dim, outputs = decoded2)
            autoencoder.compile(optimizer = 'adam', loss = 'mse')


            for k in range(series_p_auto.shape[0]):
                data_auto=np.array(series_p_auto[0]).transpose()
                autoencoder.fit(data_auto, data_auto, epochs = 10, batch_size = 2, shuffle = False) 
                encoder = Model(inputs = input_dim, outputs = encoded2)
                encoded_input = Input(shape = (encoding_dim, ))
                auto_out = np.array(encoder.predict(data_auto))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                ###^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                auto_out=pd.DataFrame(auto_out).transpose()
                encoding_dim2 = 32
                input_dim2 = Input(shape = (4, ))
    
                # Encoder Layers
                encoded12 = Dense(16, activation = 'relu')(input_dim2)
                encoded22 = Dense(32, activation = 'relu')(encoded12)
                # Decoder Layers
                decoded12 = Dense(16, activation = 'relu')(encoded22)
                decoded22 = Dense(4, activation = 'sigmoid')(decoded12)
                # Combine Encoder and Deocder layers
                autoencoder2 = Model(inputs = input_dim2, outputs = decoded22)
                autoencoder2.compile(optimizer = 'adam', loss = 'mse')
                autoencoder2.fit(auto_out, auto_out, epochs = 10, batch_size = 8, shuffle = False) 
                encoder2 = Model(inputs = input_dim2, outputs = encoded22)
                encoded_input2 = Input(shape = (encoding_dim2, ))
                auto_out = np.array(encoder2.predict(auto_out))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                ###^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                auto_out=pd.DataFrame(auto_out)
                train_pred_auto[0]=auto_out

            for k in range(series_p2_auto.shape[0]):
                data_auto=np.array(series_p2_auto[0]).transpose()
                autoencoder.fit(data_auto, data_auto, epochs = 10, batch_size = 2, shuffle = False) 
                encoder = Model(inputs = input_dim, outputs = encoded2)
                encoded_input = Input(shape = (encoding_dim, ))
                auto_out = np.array(encoder.predict(data_auto))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                ###^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                auto_out=pd.DataFrame(auto_out).transpose()
                encoding_dim2 = 32
                input_dim2 = Input(shape = (4, ))
    
                # Encoder Layers
                encoded12 = Dense(16, activation = 'relu')(input_dim2)
                encoded22 = Dense(32, activation = 'relu')(encoded12)
                # Decoder Layers
                decoded12 = Dense(16, activation = 'relu')(encoded22)
                decoded22 = Dense(4, activation = 'sigmoid')(decoded12)
                # Combine Encoder and Deocder layers
                autoencoder2 = Model(inputs = input_dim2, outputs = decoded22)
                autoencoder2.compile(optimizer = 'adam', loss = 'mse')
                autoencoder2.fit(auto_out, auto_out, epochs = 10, batch_size = 8, shuffle = False) 
                encoder2 = Model(inputs = input_dim2, outputs = encoded22)
                encoded_input2 = Input(shape = (encoding_dim2, ))
                auto_out = np.array(encoder2.predict(auto_out))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                ###^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                auto_out=pd.DataFrame(auto_out)
                val_pred_auto[0]=auto_out

            for k in range(series_p3_auto.shape[0]):
                data_auto=np.array(series_p3_auto[0]).transpose()
                autoencoder.fit(data_auto, data_auto, epochs = 10, batch_size = 2, shuffle = False) 
                encoder = Model(inputs = input_dim, outputs = encoded2)
                encoded_input = Input(shape = (encoding_dim, ))
                auto_out = np.array(encoder.predict(data_auto))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                ###^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                auto_out=pd.DataFrame(auto_out).transpose()
                encoding_dim2 = 32
                input_dim2 = Input(shape = (4, ))
    
                # Encoder Layers
                encoded12 = Dense(16, activation = 'relu')(input_dim2)
                encoded22 = Dense(32, activation = 'relu')(encoded12)
                # Decoder Layers
                decoded12 = Dense(16, activation = 'relu')(encoded22)
                decoded22 = Dense(4, activation = 'sigmoid')(decoded12)
                # Combine Encoder and Deocder layers
                autoencoder2 = Model(inputs = input_dim2, outputs = decoded22)
                autoencoder2.compile(optimizer = 'adam', loss = 'mse')
                autoencoder2.fit(auto_out, auto_out, epochs = 10, batch_size = 8, shuffle = False) 
                encoder2 = Model(inputs = input_dim2, outputs = encoded22)
                encoded_input2 = Input(shape = (encoding_dim2, ))
                auto_out = np.array(encoder2.predict(auto_out))
                auto_out=auto_out.reshape(auto_out.shape[1],)
                ###^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^	
                auto_out=pd.DataFrame(auto_out)
                test_pred_auto[0]=auto_out

################################################################################
############    Creating inputs for Neural Network      ########################
################################################################################
            #train_input = np.append(x_train,train_pred_auto,axis=1)
            train_input = x_train
            #val_input = np.append(x_val,val_pred_auto,axis=1)
            val_input = x_val
            #test_input = np.append(x_test,test_pred_auto,axis=1)
            test_input = x_test


            train_pred_auto=train_pred_auto.reshape(train_pred_auto.shape[0],32)
            val_pred_auto=val_pred_auto.reshape(val_pred_auto.shape[0],32)
            test_pred_auto=test_pred_auto.reshape(test_pred_auto.shape[0],32)

################################################################################
############       Neural Network Configuration         ########################
################################################################################

            input_data= Input(batch_shape=(None,horizon,1),name='input_data')
            input_pred=Input(batch_shape=(None,32),name='input_pred')

            branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
            branch_0=add([branch_0,input_pred])
            branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
            #branch_1=add([branch_1,input_pred])
            branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
            #branch_2=add([branch_2,input_pred])
            branch_3=Flatten()(branch_2)
            net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

            model=Model(inputs=[input_data,input_pred],outputs=net)
            callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF76.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

            model.fit({'input_data':train_input,'input_pred':train_pred_auto},y_train,validation_data=[[val_input,val_pred_auto],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
            model.load_weights('MyModelCheckpoint/CIF76.h5')
            pred=model.predict({'input_data':test_input , 'input_pred':test_pred_auto})
            pred=pred.reshape(pred.size)[:horizon]

            final_predictions[y,:horizon] = pred.reshape(horizon)
            final_test[y,:horizon]=test[-horizon:]
            pbar.update(1)
        np.savetxt('MyModelCheckpoint/prediction76.csv',final_predictions, fmt='%1.3f',delimiter=',')
        smape1=smape(final_predictions,final_test)
        mse1=mse(final_predictions,final_test)
print(datetime.datetime.now())
print("SMAPE:")
print(smape1)
print('-----------------')
print("MSE:")
print(mse1)


2019-07-29 02:56:36.012479


  0%|                                                                                           | 0/72 [00:00<?, ?it/s]

Epoch 1/10
1/1 [==============================] - 0s 229ms/step - loss: 44.1047
Epoch 2/10
1/1 [==============================] - 0s 15ms/step - loss: 44.0800
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 44.0586
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 44.0358
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 44.0107
Epoch 6/10
1/1 [==============================] - 0s 0us/step - loss: 43.9865
Epoch 7/10
1/1 [==============================] - 0s 0us/step - loss: 43.9620
Epoch 8/10
1/1 [==============================] - 0s 0us/step - loss: 43.9365
Epoch 9/10
1/1 [==============================] - 0s 763us/step - loss: 43.9110
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 43.8856
Epoch 1/10
1/1 [==============================] - 0s 248ms/step - loss: 16.0822
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 16.0289
Epoch 3/10
1/1 [==============================] - 0s 4m

Epoch 6/10
1/1 [==============================] - 0s 0us/step - loss: 42.7498
Epoch 7/10
1/1 [==============================] - 0s 0us/step - loss: 42.7299
Epoch 8/10
1/1 [==============================] - 0s 0us/step - loss: 42.7104
Epoch 9/10
1/1 [==============================] - 0s 0us/step - loss: 42.6914
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 42.6731
Epoch 1/10
1/1 [==============================] - 0s 394ms/step - loss: 15.7806
Epoch 2/10
1/1 [==============================] - 0s 0us/step - loss: 15.7031
Epoch 3/10
1/1 [==============================] - 0s 0us/step - loss: 15.6412
Epoch 4/10
1/1 [==============================] - 0s 0us/step - loss: 15.5745
Epoch 5/10
1/1 [==============================] - 0s 0us/step - loss: 15.5134
Epoch 6/10
1/1 [==============================] - 0s 0us/step - loss: 15.4578
Epoch 7/10
1/1 [==============================] - 0s 0us/step - loss: 15.4071
Epoch 8/10
1/1 [==============================] - 0s 0us/step

Epoch 1/10
1/1 [==============================] - 1s 547ms/step - loss: 16.1224
Epoch 2/10
1/1 [==============================] - 0s 0us/step - loss: 16.0688
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 16.0087
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 15.9520
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 15.9239
Epoch 6/10
1/1 [==============================] - 0s 0us/step - loss: 15.8955
Epoch 7/10
1/1 [==============================] - 0s 4ms/step - loss: 15.8663
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 15.8365
Epoch 9/10
1/1 [==============================] - 0s 4ms/step - loss: 15.8066
Epoch 10/10
1/1 [==============================] - 0s 0us/step - loss: 15.7760
Epoch 1/10
1/1 [==============================] - 0s 0us/step - loss: 42.2111
Epoch 2/10
1/1 [==============================] - 0s 0us/step - loss: 42.2058
Epoch 3/10
1/1 [==============================] - 0s 0us/step

...

 12%|█████████▎                                                                | 9/72 [10:08:49<114:16:15, 6529.77s/it]

Epoch 1/10
1/1 [==============================] - 70s 70s/step - loss: 5.6123
Epoch 2/10
1/1 [==============================] - 0s 31ms/step - loss: 5.6062
Epoch 3/10
1/1 [==============================] - 0s 31ms/step - loss: 5.6001
Epoch 4/10
1/1 [==============================] - 0s 47ms/step - loss: 5.5941
Epoch 5/10
1/1 [==============================] - 0s 31ms/step - loss: 5.5881
Epoch 6/10
1/1 [==============================] - 0s 47ms/step - loss: 5.5825
Epoch 7/10
1/1 [==============================] - 0s 31ms/step - loss: 5.5766
Epoch 8/10
1/1 [==============================] - 0s 31ms/step - loss: 5.5709
Epoch 9/10
1/1 [==============================] - 0s 47ms/step - loss: 5.5653
Epoch 10/10
1/1 [==============================] - 0s 31ms/step - loss: 5.5596
Epoch 1/10
1/1 [==============================] - 71s 71s/step - loss: 2.1092
Epoch 2/10
1/1 [==============================] - 0s 47ms/step - loss: 2.0974
Epoch 3/10
1/1 [==============================] - 0s 31ms/step 

Epoch 6/10
1/1 [==============================] - 0s 34ms/step - loss: 5.3340
Epoch 7/10
1/1 [==============================] - 0s 41ms/step - loss: 5.3286
Epoch 8/10
1/1 [==============================] - 0s 53ms/step - loss: 5.3233
Epoch 9/10
1/1 [==============================] - 0s 54ms/step - loss: 5.3178
Epoch 10/10
1/1 [==============================] - 0s 37ms/step - loss: 5.3123
Epoch 1/10
1/1 [==============================] - 79s 79s/step - loss: 2.2400
Epoch 2/10
1/1 [==============================] - 0s 41ms/step - loss: 2.2119
Epoch 3/10
1/1 [==============================] - 0s 41ms/step - loss: 2.1884
Epoch 4/10
1/1 [==============================] - 0s 38ms/step - loss: 2.1614
Epoch 5/10
1/1 [==============================] - 0s 29ms/step - loss: 2.1365
Epoch 6/10
1/1 [==============================] - 0s 42ms/step - loss: 2.1106
Epoch 7/10
1/1 [==============================] - 0s 34ms/step - loss: 2.0871
Epoch 8/10
1/1 [==============================] - 0s 41ms/step 

Epoch 1/10
1/1 [==============================] - 77s 77s/step - loss: 7.0060
Epoch 2/10
1/1 [==============================] - 0s 43ms/step - loss: 6.9741
Epoch 3/10
1/1 [==============================] - 0s 42ms/step - loss: 6.9469
Epoch 4/10
1/1 [==============================] - 0s 48ms/step - loss: 6.9201
Epoch 5/10
1/1 [==============================] - 0s 22ms/step - loss: 6.9068
Epoch 6/10
1/1 [==============================] - 0s 27ms/step - loss: 6.8932
Epoch 7/10
1/1 [==============================] - 0s 31ms/step - loss: 6.8794
Epoch 8/10
1/1 [==============================] - 0s 51ms/step - loss: 6.8655
Epoch 9/10
1/1 [==============================] - 0s 28ms/step - loss: 6.8516
Epoch 10/10
1/1 [==============================] - 0s 35ms/step - loss: 6.8377
Epoch 1/10
1/1 [==============================] - 0s 35ms/step - loss: 4.8019
Epoch 2/10
1/1 [==============================] - 0s 38ms/step - loss: 4.7847
Epoch 3/10
1/1 [==============================] - 0s 25ms/step 

1/1 [==============================] - 0s 23ms/step - loss: 23.2111
Epoch 6/10
1/1 [==============================] - 0s 25ms/step - loss: 23.1203
Epoch 7/10
1/1 [==============================] - 0s 32ms/step - loss: 23.0290
Epoch 8/10
1/1 [==============================] - 0s 37ms/step - loss: 22.9355
Epoch 9/10
1/1 [==============================] - 0s 22ms/step - loss: 22.8442
Epoch 10/10
1/1 [==============================] - 0s 32ms/step - loss: 22.7534
Epoch 1/10
1/1 [==============================] - 0s 27ms/step - loss: 3.8414
Epoch 2/10
1/1 [==============================] - 0s 33ms/step - loss: 3.8279
Epoch 3/10
1/1 [==============================] - 0s 37ms/step - loss: 3.8149
Epoch 4/10
1/1 [==============================] - 0s 31ms/step - loss: 3.8024
Epoch 5/10
1/1 [==============================] - 0s 25ms/step - loss: 3.7903
Epoch 6/10
1/1 [==============================] - 0s 37ms/step - loss: 3.7786
Epoch 7/10
1/1 [==============================] - 0s 36ms/step - los

In [ ]:
2+2